# Time Series Anomaly Detection Exercises
### Kwame V. Taylor


* Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? What are two hypotheses you can test? Use time series method for detecting anomalies, like exponential moving average with %b.

**Bonus:**
 * Can you label students who are viewing both the web dev and data science curriculum?
 * Can you label students by the program they are in?
 * Can you label users by student vs. staff?
 * What are Zach, Maggie, Faith, and Ryan's ids?

### Imports

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

### Acquire

In [2]:
colnames = ['date', 'timestamp', 'request_method', 'user_id', 'cohort_id', 'ip']

df = pd.read_csv('anonymized-curriculum-access.txt', header=None, index_col=False,
                 names=colnames, delim_whitespace=True, na_values='"-"')
#                 usecols=[0, 1, 2, 5])
df.head()

,date,timestamp,request_method,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61


### Prepare

In [3]:
# merge date and timestamp
df["ds"] = df["date"] +" "+ df["timestamp"]
df.head()

,date,timestamp,request_method,user_id,cohort_id,ip,ds
0,2018-01-26,09:55:03,/,1,8,97.105.19.61,2018-01-26 09:55:03
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61,2018-01-26 09:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,2018-01-26 09:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,2018-01-26 09:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61,2018-01-26 09:56:24


In [4]:
# drop date and timestamp
df = df.drop(columns=['date', 'timestamp'])
df.head()

,request_method,user_id,cohort_id,ip,ds
0,/,1,8,97.105.19.61,2018-01-26 09:55:03
1,java-ii,1,8,97.105.19.61,2018-01-26 09:56:02
2,java-ii/object-oriented-programming,1,8,97.105.19.61,2018-01-26 09:56:05
3,slides/object_oriented_programming,1,8,97.105.19.61,2018-01-26 09:56:06
4,javascript-i/conditionals,2,22,97.105.19.61,2018-01-26 09:56:24


In [5]:
# convert date column to datetime type
df.ds = pd.to_datetime(df.ds)
df.dtypes

request_method            object
user_id                    int64
cohort_id                 object
ip                        object
ds                datetime64[ns]
dtype: object

In [6]:
# set ds as index and sort
# this is a very important step!
df = df.set_index('ds').sort_index()
df.head()

,request_method,user_id,cohort_id,ip
ds,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61


In [7]:
df.shape

(719459, 4)

In [13]:
df.request_method.value_counts()

/                                36159
javascript-i                     13894
search/search_index.json         13582
toc                              13379
java-iii                         10590
                                 ...  
examples/css/..%5c                   1
4.2-compare-means                    1
content/control-structures-ii        1
115                                  1
codeup-exercises                     1
Name: request_method, Length: 2060, dtype: int64

In [9]:
df.ip.value_counts()

97.105.19.58       268648
97.105.19.61        60530
192.171.117.210      8896
71.150.217.33        4919
12.106.208.194       4262
                    ...  
70.125.211.106          1
99.203.155.5            1
184.203.98.134          1
107.77.198.150          1
107.77.221.18           1
Name: ip, Length: 4064, dtype: int64

In [12]:
df.cohort_id.value_counts()

28    60145
33    40151
29    37401
53    36039
24    35615
57    32397
56    31652
22    30309
51    29668
23    28310
32    28310
58    28250
26    27625
52    27513
25    25423
34    25176
31    25163
59    22345
27    20414
55    20400
61    11631
14     9487
1      8879
62     8666
21     7443
17     4917
13     2733
18     2003
8      1710
19     1165
16      742
15      691
7       495
12      302
11      208
2        93
6        72
9         5
4         4
5         1
Name: cohort_id, dtype: int64

Some IP addresses accidentally got put in the ```cohort_id``` column, so I'm going to drop those rows.

In [11]:
df = df.drop(df[df.cohort_id.str.len() > 4].index)
df

,request_method,user_id,cohort_id,ip
ds,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61
...,...,...,...,...
2020-11-02 16:48:13,javascript-i/coffee-project,763,62,107.192.148.199
2020-11-02 16:48:17,javascript-i/mapbox-api,771,62,172.125.226.175
2020-11-02 16:48:18,javascript-i/coffee-project,771,62,172.125.226.175


In [ ]:
#from ryan
#mask = (df.column.str.contains(“A”) | df.column.str.contains(“B”))
#df[mask]

In [14]:
df.isna().sum()

request_method      0
user_id             0
cohort_id         420
ip                  0
dtype: int64

In [16]:
df = df.dropna()

In [ ]:
**S

### Explore

In [19]:
df.ip.value_counts().sum()

673553

In [20]:
df.ip.value_counts()

97.105.19.58       268254
97.105.19.61        60492
192.171.117.210      8894
71.150.217.33        4913
12.106.208.194       4261
                    ...  
96.88.69.9              1
107.77.197.78           1
172.58.107.4            1
174.197.4.51            1
107.77.221.18           1
Name: ip, Length: 4047, dtype: int64

In [33]:
sub_df = df.groupby('ip').filter(lambda x : len(x)>6)
sub_df.ip.value_counts()

97.105.19.58       268254
97.105.19.61        60492
192.171.117.210      8894
71.150.217.33        4913
12.106.208.194       4261
                    ...  
173.239.232.160         7
99.203.27.103           7
172.58.67.201           7
70.243.68.60            7
99.203.26.254           7
Name: ip, Length: 2030, dtype: int64

In [34]:
sub_df.shape

(667916, 4)

In [26]:
(df.drop(df[df.ip.value_counts() > 4].index)).sum()

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
ss = "Sammy Shark!"
print(ss[4])

### Hypothesis 1:
    
    